In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import Button, HBox, VBox, Output, Layout
from IPython.display import display, clear_output

input_folder = '/Users/rfalcao/Documents/ClearCuspsCroppedRotated'  
output_folder = "/Users/rfalcao/Documents/Segmented Cusps trial"  
os.makedirs(output_folder, exist_ok=True)

image_files = [os.path.join(input_folder, f) for f in os.listdir(input_folder) if f.lower().endswith(('.png', '.jpg', '.tif'))]
image_queue = image_files[:]  
small_image_queue = []  

output = Output()

def update_display(image, title):
    with output:
        clear_output(wait=True)
        plt.figure(figsize=(6, 6))
        plt.title(title)
        plt.imshow(image, cmap='gray')
        plt.axis('off')
        plt.show()

# Function to split the image into several 250x250 regions
def split_image(image, size=250):
    h, w = image.shape
    sub_images = []
    for y in range(0, h - size + 1, size):  # sliding window approach (can adjust step size)
        for x in range(0, w - size + 1, size):
            sub_images.append(image[y:y+size, x:x+size])
    return sub_images

# Function to process next image
def next_image():
    if small_image_queue:
        # Process the most recent small image first
        s_image, base_name, i = small_image_queue[-1]
        update_display(s_image, f"Image part {i} of {base_name}")
    elif image_queue:
        # Process the next image in the original queue
        image_path = image_queue[0]
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        update_display(image, f"Inspecting: {os.path.basename(image_path)}")
    else:
        with output:
            clear_output(wait=True)
            print("All images processed!")

# Button callbacks
def on_yes_clicked(b):
    if small_image_queue:
        # Process the most recent image part
        s_image, base_name, i = small_image_queue.pop()
        if s_image.shape[0] == 250 and s_image.shape[1] == 250:  # Only save 250x250 images
            save_path = os.path.join(output_folder, f"{base_name}_part_{i}.png")
            cv2.imwrite(save_path, s_image)  
            print(f"Saved: {save_path}")
    elif image_queue:
        # Split the next image in the original queue
        image_path = image_queue.pop(0)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        base_name = os.path.splitext(os.path.basename(image_path))[0]
        sub_images = split_image(image)
        for i, s in enumerate(sub_images):
            small_image_queue.append((s, base_name, i))  # Add sub-images to the queue
    next_image()  

def on_no_clicked(b):
    if small_image_queue:
        small_image_queue.pop() 
    next_image()  

# Create buttons
yes_button = Button(description="Yes (Save part)", button_style='success', layout=Layout(width='150px'))
no_button = Button(description="No (Skip)", button_style='warning', layout=Layout(width='150px'))

yes_button.on_click(on_yes_clicked)
no_button.on_click(on_no_clicked)

controls = HBox([yes_button, no_button])
display(VBox([output, controls]))

# Start processing
next_image()
